In [1]:
import folium
import geopandas as gpd

from pystac_client import Client

In [2]:
# open static STAC catalog
catalog = Client.open("data/catalog.json")
catalog.title

's1s2_water'

In [3]:
# get all items and convert them to geopandas dataframe
items = [item.to_dict() for item in catalog.get_all_items()]
gdf = gpd.GeoDataFrame.from_features(features=items, crs=3857).to_crs(4326)
gdf.head()

,geometry,flood,split,datetime,landcover,s1_srcids,s2_srcids,copdem30_slope,copdem30_elevation,date_s1,date_s2
0,"POLYGON ((166.27203 -45.14621, 167.66760 -45.1...",False,val,2020-01-01T00:00:00Z,"Tree cover, broadleaved, evergreen, closed to ...",[S1B_IW_GRDH_1SDV_20200206T075413_20200206T075...,[S2B_MSIL1C_20200209T224759_N0209_R115_T58GFQ_...,sentinel12_copdem30_93_slope,sentinel12_copdem30_93_elevation,20200206,20200209
1,"POLYGON ((106.16716 39.74440, 107.44788 39.724...",False,train,2020-01-01T00:00:00Z,Grassland,[S1A_IW_GRDH_1SDV_20201108T104645_20201108T104...,[S2A_MSIL1C_20201107T033941_N0209_R061_T48SXJ_...,sentinel12_copdem30_91_slope,sentinel12_copdem30_91_elevation,20201108,20201107
2,"POLYGON ((-14.09034 8.95716, -14.09034 9.94593...",False,train,2020-01-01T00:00:00Z,Water bodies,[S1A_IW_GRDH_1SDV_20190422T190812_20190422T190...,[S2A_MSIL1C_20190422T110621_N0207_R137_T28PFR_...,sentinel12_copdem30_9_slope,sentinel12_copdem30_9_elevation,20190422,20190422
3,"POLYGON ((50.99980 25.31673, 52.09064 25.31270...",False,test,2020-01-01T00:00:00Z,Water bodies,[S1A_IW_GRDH_1SDV_20200410T023157_20200410T023...,[S2A_MSIL1C_20200414T070621_N0209_R106_T39RWH_...,sentinel12_copdem30_89_slope,sentinel12_copdem30_89_elevation,20200410,20200414
4,"POLYGON ((32.02016 23.50747, 33.09560 23.51053...",False,test,2020-01-01T00:00:00Z,Bare areas,[S1B_IW_GRDH_1SDV_20201124T155420_20201124T155...,[S2B_MSIL1C_20201126T082259_N0209_R121_T36QVL_...,sentinel12_copdem30_88_slope,sentinel12_copdem30_88_elevation,20201124,20201126


In [4]:
# visualize samples on map (train, val, test)
m = folium.Map(tiles="Stamen Terrain")

popup = folium.GeoJsonPopup(
    fields=["split", "landcover"],
    aliases=["split", "landcover"],
    sticky=True,
    labels=True,
    style="background-color: #F0EFEF; border: 1px solid black; border-radius: 3px; box-shadow: 3px;",
)


def style_function(feature):
    return {
        "fillOpacity": 0.9,
        "weight": 0,
        "fillColor": "#00ff00"
        if feature["properties"]["split"] == "train"
        else "#0000ff"
        if feature["properties"]["split"] == "val"
        else "#ff0000",
    }


folium.GeoJson(gdf.to_json(), popup=popup, style_function=style_function).add_to(m)
m.fit_bounds(m.get_bounds())
m
